In [1]:
import DCCRN
import torch
from asteroid.data import DNSDataset
import os
import time
import torch
import shutil
import numpy as np

import asteroid
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import config as cfg
from tools_for_model import near_avg_index, max_index, min_index, Bar, cal_pesq, cal_stoi

import inspect


/opt/conda/lib/python3.10/site-packages/asteroid/filterbanks/__init__.py:24: VisibleDeprecationWarning: asteroid.filterbanks has been replaced by `asteroid_filterbanks` and will be totally remove in a future release. Please use `asteroid_filterbanks` instead.
  warnings.warn(


In [2]:
model = DCCRN.DCCRN()
data_set =  DNSDataset("/root/NTH_student/asteroid/egs/dns_challenge/baseline/data")

In [3]:
# tạo sample để input mode. cái này random đại thui
input_size = model.stft.weight.size(1)
# tạo cái batch có n= 5 đi
input = torch.randn(5, input_size)

In [4]:
# Tạo list chứa các feature map do encoder output ra 
# dùng list vì lát nữa feature map của lstm với decoder bỏ vô đây luôn. ở đây demo cái encoder thui nha. 
feature_maps = []
# Tạo hàm hook, hàm này sẽ thực thi khi encoder thực hiện forward.
# output của encoder sẽ được append vào trong encoder_feature_map(một biến toàn cục)
def encoder_hook(module, input, output):
    feature_maps.append(output)
# đăng kí forward hook. kiểu "ê khi nào encoder thực hiện foward xong thì chạy hàm encoder_hook nha"
model.encoder[-1].register_forward_hook(encoder_hook)

In [5]:
try :
    model(input)
    print('au de')
except:
    print('failed roihuhu')

au de


/root/NTH_student/knowledge_distillation_CLSKD/tools_for_model.py:472: UserWarning: This overload of addcmul is deprecated:
	addcmul(Tensor input, Number value, Tensor tensor1, Tensor tensor2, *, Tensor out)
Consider using one of the following signatures instead:
	addcmul(Tensor input, Tensor tensor1, Tensor tensor2, *, Number value, Tensor out) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1174.)
  delta = torch.addcmul(Vrr * Vii, -1, Vri, Vri)


In [6]:
print(len(feature_maps))

1


In [7]:
feature_maps[0].shape

torch.Size([5, 256, 4, 3])

In [8]:
feature_maps

[tensor([[[[-1.2585e-01,  2.6200e-02,  1.1350e+00],
           [-9.3977e-02,  2.6328e-01,  4.7110e-01],
           [ 8.6911e-02,  6.8840e-02,  1.3020e+00],
           [-3.6539e-02,  4.4229e-01, -1.5921e-01]],
 
          [[-4.7572e-02,  3.0941e-01,  1.8745e-01],
           [ 5.3147e-02,  1.0413e+00,  1.0343e+00],
           [ 9.5400e-02,  3.8757e-01,  5.0434e-01],
           [ 6.9664e-02,  1.0060e+00,  1.1015e+00]],
 
          [[-6.9156e-02,  4.0085e-02, -1.9666e-01],
           [ 6.8443e-01,  4.3281e-01,  6.8128e-01],
           [-5.3679e-02,  1.1325e+00, -4.7670e-01],
           [ 2.6338e-01,  4.2350e-01,  1.8954e-01]],
 
          ...,
 
          [[-2.0606e-01, -1.6712e-01, -2.4150e-01],
           [-1.6242e-01,  1.1270e-01, -6.2741e-02],
           [-1.0988e-01, -8.9376e-02, -2.0386e-01],
           [-1.4564e-01, -1.0669e-01, -6.4342e-02]],
 
          [[ 4.3390e-01, -2.4586e-01,  1.3781e-01],
           [ 7.6600e-01, -3.3203e-01, -1.5774e-01],
           [ 7.4555e-01, -3.1676e-0

In [44]:
feature_maps.clear()